In [ ]:
# Import libraries. You may or may not use all of these.

!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data

!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
data_for_visual_analysis = dataset.copy()

Visualization of the correlation between expenses and other data groups.

In [ ]:
# Age groups

bins = [0, 20, 30, 40, 50, 60, 70, 80]
labels = ['0-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-100']
data_for_visual_analysis['age_group'] = pd.cut(data_for_visual_analysis['age'], bins=bins, labels=labels)


# Bar plot of Average Expenses by Age Group

average_expenses_age_group = data_for_visual_analysis.groupby('age_group')['expenses'].mean().reset_index()
plt.figure(figsize=(8, 5))
plt.bar(average_expenses_age_group['age_group'], average_expenses_age_group['expenses'], color='orange')
plt.title('Average Expenses by Age Group')
plt.xlabel('Age Group')
plt.ylabel('Average Expenses')
plt.grid(axis='y')
plt.show()

In [ ]:
# Bar plot of Average Expenses by Sex

expenses_by_sex = data_for_visual_analysis.groupby('sex')['expenses'].mean()
plt.figure(figsize=(8, 5))
expenses_by_sex.plot(kind='bar', color=['orange', 'skyblue'])
plt.title('Average Expenses by Sex')
plt.xlabel('Sex')
plt.ylabel('Average Expenses')
plt.xticks(rotation=0)
plt.grid(axis='y')
plt.show()

In [ ]:
# BMI groups

bins = [0, 20, 25, 30, 35, 100]
labels = ['<20', '21-25', '26-30', '31-35', '>35']
data_for_visual_analysis['bmi_group'] = pd.cut(data_for_visual_analysis['bmi'], bins=bins, labels=labels, right=False)


# Bar plot of Average Expenses by BMI Group

average_expenses_bmi_group = data_for_visual_analysis.groupby('bmi_group')['expenses'].mean().reset_index()
plt.figure(figsize=(8, 5))
plt.bar(average_expenses_bmi_group['bmi_group'], average_expenses_bmi_group['expenses'], color='orange')
plt.title('Average Expenses by BMI Group')
plt.xlabel('BMI Group')
plt.ylabel('Average Expenses')
plt.grid(axis='y')
plt.show()

In [ ]:
# Bar plot of Average Expenses by Smoker Status

expenses_by_smoker = data_for_visual_analysis.groupby('smoker')['expenses'].mean()
plt.figure(figsize=(8, 5))
expenses_by_smoker.plot(kind='bar', color=['orange', 'skyblue'])
plt.title('Average Expenses by Smoker Status')
plt.xlabel('Smoker Status')
plt.ylabel('Average Expenses')
plt.xticks(rotation=0)
plt.grid(axis='y')
plt.show()

In [ ]:
# Bar plot of Average Expenses by Number of Children

expenses_by_number_of_children = data_for_visual_analysis.groupby('children')['expenses'].mean()
plt.figure(figsize=(8, 5))
expenses_by_number_of_children.plot(kind='bar', color='orange')
plt.title('Average Expenses by Number of Children')
plt.xlabel('Number of Children')
plt.ylabel('Average Expenses')
plt.xticks(rotation=0)
plt.grid(axis='y')
plt.show()

In [ ]:
# Bar plot of Average Expenses by Region

average_expenses_region = data_for_visual_analysis.groupby('region')['expenses'].mean().reset_index()
plt.figure(figsize=(8, 5))
plt.bar(average_expenses_region['region'], average_expenses_region['expenses'], color='skyblue')
plt.title('Average Expenses by Region')
plt.xlabel('Region')
plt.ylabel('Average Expenses')
plt.grid(axis='y')
plt.show()

In [ ]:
data_for_model = dataset.copy()

In [ ]:
# Convert categorical data to numbers

data_for_model['smoker'] = data_for_model['smoker'].apply(lambda x: 1 if x == 'yes' else 0)

data_for_model['sex'] = data_for_model['sex'].apply(lambda x: 1 if x == 'female' else 0)

unique_regions = data_for_model['region'].unique()
for region in unique_regions:
    data_for_model[region] = data_for_model['region'].apply(lambda x: 1 if x == region else 0)
data_for_model.drop(columns=['region'], inplace=True)

In [ ]:
# Separate target data and features

target = data_for_model.pop('expenses')
features = data_for_model

In [ ]:
# Separate data into train_dataset and and test_dataset

shuffled_indices = np.arange(features.shape[0])
np.random.shuffle(shuffled_indices)
split_index = int(0.8 * len(shuffled_indices))

train_indices = shuffled_indices[:split_index]
test_indices = shuffled_indices[split_index:]

train_dataset = features.iloc[train_indices]
test_dataset = features.iloc[test_indices]
train_labels = target.iloc[train_indices]
test_labels = target.iloc[test_indices]

In [ ]:
# Build and train the regression model

model = tf.keras.Sequential()
model.add(layers.InputLayer(input_shape=(features.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1))

model.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.05, rho=0.9),
    loss='mae',
    metrics=['mae', 'mse']
)

model.fit(
    train_dataset,
    train_labels,
    epochs=60,
    shuffle=True,
    verbose="auto"
)

In [ ]:
model.evaluate(
    test_dataset,
    test_labels,
    verbose='auto'
)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
